##SCD TYPE-1##

####In SCD Type-1 Only changed are reflected into the Targeted Table ####

In [0]:
%sql
create table datamodeling.default.scdtyp1_source
(
  prod_id int,
  prod_name string,
  prod_cat string,
  processDate date
)


In [0]:
%sql
insert into datamodeling.default.scdtyp1_source
values
(1,'prod1','cat1',current_date()),
(2,'prod2','cat2',current_date()),
(3,'prod3','cat3',current_date()),
(4,'prod4','cat4',current_date()),
(5,'prod5','cat5',current_date())


###Creating Target Table###

In [0]:
%sql
create table datamodeling.gold.scdtyp1_table
(
 prod_id int,
 prod_name string,
 prod_cat string,
 processDate date 
)

In [0]:
spark.sql("select * from datamodeling.default.scdtyp1_source").createOrReplaceTempView("src")

###Merg into the changed tgt and Back-Dated rows as well###

In [0]:
%sql
merge into datamodeling.gold.scdtyp1_table t
using datamodeling.default.scdtyp1_source s
on t.prod_id = s.prod_id
when matched and s.processDate >= t.processDate then update set * --s.processDate > t.processDate is for back dated refresh
when not matched then insert *

In [0]:
%sql
select * from datamodeling.gold.scdtyp1_table

###Update a column in source scdtyp1###

In [0]:
%sql
update datamodeling.default.scdtyp1_source
set prod_cat ='newCategory' where prod_id=3

In [0]:
%sql
select * from datamodeling.default.scdtyp1_source

In [0]:
%sql
merge into datamodeling.gold.scdtyp1_table t
using datamodeling.default.scdtyp1_source s
on t.prod_id = s.prod_id
when matched and s.processDate >= t.processDate then update set * --s.processDate > t.processDate is for back dated refresh
when not matched then insert *

In [0]:
%sql
select * from datamodeling.gold.scdtyp1_table

##SCD Type-2##

####In SCD Type-2 Keep the track of changing records and keep the changed record as well ####

###Create a Src for the SCD-Type-2###

In [0]:
%sql
create or replace table datamodeling.default.scdtyp2_source
(
 prod_id int,
 prod_name string,
 prod_cat string,
 processDate date 
)


In [0]:
%sql
insert into datamodeling.default.scdtyp2_source
values
(1,'prod1','cat1',current_date()),
(2,'prod2','cat2',current_date()),
(3,'prod3','cat3',current_date()),
(4,'prod4','cat4',current_date()),
(5,'prod5','cat5',current_date())

###Creating Destination Table in Gold###

In [0]:
%sql
create or replace table datamodeling.gold.scdtyp2_table(
  prod_id int,
  prod_name string,
  prod_cat string,
  processDate date,
  start_date date,
  end_date date,
  is_current string
)

###In Destination few Column are added###

In [0]:
%sql
select *, current_timestamp as start_date, cast('3000-01-01' as timestamp) as end_date, 'Y' as is_current from datamodeling.default.scdtyp2_source

###creating source###

In [0]:
spark.sql("select *, current_timestamp as start_date, cast('3000-01-01' as timestamp) as end_date, 'Y' as is_current from datamodeling.default.scdtyp2_source").createOrReplaceTempView("src")

###Creating Merge for SCD-Type-2###

In [0]:
%sql
merge into datamodeling.gold.scdtyp2_table t
using src 
on src.prod_id = t.prod_id
and t.is_current = 'Y'
-- When we have new Data with Updates
when matched and (
  src.prod_cat<>t.prod_cat or
  src.prod_name<>t.prod_name or 
  src.processDate<>t.processDate
) then update set
  t.end_date = current_timestamp(),
  t.is_current = 'N'




In [0]:
%sql
select * from datamodeling.gold.scdtyp2_table

In [0]:
%sql
merge into datamodeling.gold.scdtyp2_table t
using src 
on t.prod_id = src.prod_id and 
t.is_current = 'Y'
when matched and src.is_current = 'N' then update set t.is_current = 'N'
when not matched then insert (prod_id, prod_name, prod_cat, processDate, start_date, end_date, is_current) values (src.prod_id, src.prod_name, src.prod_cat, src.processDate,src.start_date,src.end_date,src.is_current)


In [0]:
%sql
select * from datamodeling.gold.scdtyp2_table

###Update the record in SCDType2-source###

In [0]:
%sql
update datamodeling.default.scdtyp2_source set prod_cat = 'smartCategory' where prod_id = 2